**Preparing the Data to do in Mode**

In [1]:
import pandas as pd
import numpy as np
import scipy as sc
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import joblib
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
import time


In [170]:
carSales =pd.read_csv("../Milestone1/Car_sales_CleanData.csv")
carSales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23090 entries, 0 to 23089
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Date           23090 non-null  object
 1   Customer Name  23090 non-null  object
 2   Gender         23090 non-null  object
 3   Annual Income  23090 non-null  int64 
 4   Dealer_Name    23090 non-null  object
 5   Company        23090 non-null  object
 6   Model          23090 non-null  object
 7   Engine         23090 non-null  object
 8   Transmission   23090 non-null  object
 9   Color          23090 non-null  object
 10  Price ($)      23090 non-null  int64 
 11  Dealer_No      23090 non-null  object
 12  Body Style     23090 non-null  object
 13  Phone          23090 non-null  int64 
 14  Dealer_Region  23090 non-null  object
 15  Year           23090 non-null  int64 
 16  Month          23090 non-null  int64 
 17  DayOfWeek      23090 non-null  int64 
 18  Season         23090 non-n

In [171]:
carSales = carSales.drop(columns=['Customer Name', 'Dealer_Name', 'Dealer_No ', 'Phone'])

In [172]:
carSales.info()
carSales.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23090 entries, 0 to 23089
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Date           23090 non-null  object
 1   Gender         23090 non-null  object
 2   Annual Income  23090 non-null  int64 
 3   Company        23090 non-null  object
 4   Model          23090 non-null  object
 5   Engine         23090 non-null  object
 6   Transmission   23090 non-null  object
 7   Color          23090 non-null  object
 8   Price ($)      23090 non-null  int64 
 9   Body Style     23090 non-null  object
 10  Dealer_Region  23090 non-null  object
 11  Year           23090 non-null  int64 
 12  Month          23090 non-null  int64 
 13  DayOfWeek      23090 non-null  int64 
 14  Season         23090 non-null  object
dtypes: int64(5), object(10)
memory usage: 2.6+ MB


(23090, 15)

# Feature Engineering

In [173]:
import pandas as pd
from datetime import datetime
import holidays
from dateutil.relativedelta import relativedelta, FR

# Egyptian holidays in English
eg_holidays = holidays.Egypt(language='en')

# Function to get Black Friday (last Friday in November)
def get_black_friday(year):
    last_day = datetime(year, 11, 30)
    return last_day + relativedelta(weekday=FR(-1))

# Function to return fixed custom holidays
def get_custom_holidays(year):
    return {
        "Christmas": datetime(year, 12, 25),
        "Valentine's Day": datetime(year, 2, 14),
        "Mother's Day": datetime(year, 3, 21),
        "Black Friday": get_black_friday(year)
    }

# Function to return the name of the holiday (either from holidays package or custom)
def get_holiday_name(date):
    date = pd.to_datetime(date).date()
    
    # Check official holidays
    if date in eg_holidays:
        return eg_holidays.get(date)

    # Check custom holidays
    for name, d in get_custom_holidays(date.year).items():
        if d.date() == date:
            return name

    return None


In [174]:
carSales['Price_to_Income'] = carSales['Price ($)'] / carSales['Annual Income']

In [175]:
carSales['Holiday'] = carSales['Date'].apply(get_holiday_name)

In [176]:
carSales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23090 entries, 0 to 23089
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             23090 non-null  object 
 1   Gender           23090 non-null  object 
 2   Annual Income    23090 non-null  int64  
 3   Company          23090 non-null  object 
 4   Model            23090 non-null  object 
 5   Engine           23090 non-null  object 
 6   Transmission     23090 non-null  object 
 7   Color            23090 non-null  object 
 8   Price ($)        23090 non-null  int64  
 9   Body Style       23090 non-null  object 
 10  Dealer_Region    23090 non-null  object 
 11  Year             23090 non-null  int64  
 12  Month            23090 non-null  int64  
 13  DayOfWeek        23090 non-null  int64  
 14  Season           23090 non-null  object 
 15  Price_to_Income  23090 non-null  float64
 16  Holiday          1248 non-null   object 
dtypes: float64(1

In [177]:
carSales.Holiday.value_counts()

Holiday
Christmas                                     169
(تقدير) عطلة عيد الفطر                        167
Black Friday                                  139
(تقدير) عيد الأضحى                            113
(تقدير) عطلة عيد الأضحى                        67
عيد ثورة 23 يوليو                              65
عيد العمال                                     63
(تقدير) رأس السنة الهجرية                      63
Valentine's Day                                59
Mother's Day                                   45
عيد الفصح القبطي                               40
(تقدير) عيد المولد النبوي                      39
(تقدير) عيد الفطر                              38
شم النسيم; عيد تحرير سيناء                     35
عيد ثورة 25 يناير                              30
شم النسيم                                      30
عيد القوات المسلحة                             23
(تقدير) يوم عرفة                               20
عيد الميلاد المجيد (تقويم قبطي)                15
عيد تحرير سيناء                           

In [178]:
carSales['Is_Holiday'] = carSales['Date'].apply(lambda x: 1 if get_holiday_name(x) else 0)

In [179]:
carSales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23090 entries, 0 to 23089
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             23090 non-null  object 
 1   Gender           23090 non-null  object 
 2   Annual Income    23090 non-null  int64  
 3   Company          23090 non-null  object 
 4   Model            23090 non-null  object 
 5   Engine           23090 non-null  object 
 6   Transmission     23090 non-null  object 
 7   Color            23090 non-null  object 
 8   Price ($)        23090 non-null  int64  
 9   Body Style       23090 non-null  object 
 10  Dealer_Region    23090 non-null  object 
 11  Year             23090 non-null  int64  
 12  Month            23090 non-null  int64  
 13  DayOfWeek        23090 non-null  int64  
 14  Season           23090 non-null  object 
 15  Price_to_Income  23090 non-null  float64
 16  Holiday          1248 non-null   object 
 17  Is_Holiday  

#

# **duplicates**

In [180]:
def check_duplicates(df):
    if df.duplicated().sum() > 0:
        print(f"Duplicate rows found: {df.duplicated().sum()}")
        return True
    return False

In [181]:
carSales = carSales.drop_duplicates()
check_duplicates(carSales)

False

# **data types**

In [182]:
def check_data_types(df):
    for col in df.columns:
        if df[col].dtype not in ['int64', 'float64']:
            print(f"Column '{col}' has data type '{df[col].dtype}'")
            return True
    return False
check_data_types(carSales)

Column 'Date' has data type 'object'


True

In [183]:
# Convert all boolean columns to 0 and 1
bool_columns = carSales.select_dtypes(include=['bool']).columns  # Get all bool columns
carSales[bool_columns] = carSales[bool_columns].astype(int)

In [184]:
# Convert 'Date' to datetime
carSales['Date'] = pd.to_datetime(carSales['Date'])



In [185]:
carSales['Year'] = pd.to_datetime(carSales['Date']).dt.year
carSales['Month'] = pd.to_datetime(carSales['Date']).dt.month
carSales['Day'] = pd.to_datetime(carSales['Date']).dt.day
carSales['DayOfWeek'] = pd.to_datetime(carSales['Date']).dt.dayofweek
carSales['WeekOfYear'] = carSales['Date'].dt.isocalendar().week
carSales['Is_Weekend'] = carSales['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)


In [186]:
carSales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23089 entries, 0 to 23089
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             23089 non-null  datetime64[ns]
 1   Gender           23089 non-null  object        
 2   Annual Income    23089 non-null  int64         
 3   Company          23089 non-null  object        
 4   Model            23089 non-null  object        
 5   Engine           23089 non-null  object        
 6   Transmission     23089 non-null  object        
 7   Color            23089 non-null  object        
 8   Price ($)        23089 non-null  int64         
 9   Body Style       23089 non-null  object        
 10  Dealer_Region    23089 non-null  object        
 11  Year             23089 non-null  int32         
 12  Month            23089 non-null  int32         
 13  DayOfWeek        23089 non-null  int32         
 14  Season           23089 non-null  object    

# **missing values**

In [187]:
def check_missing_values(df):
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            print(f"Missing values in column '{col}': {df[col].isnull().sum()}")
            return True
    return False
check_missing_values(carSales)

Missing values in column 'Holiday': 21841


True

In [188]:
carSales['Holiday'].fillna('No Holiday', inplace=True)
print(carSales['Holiday'].value_counts())

Holiday
No Holiday                                    21841
Christmas                                       169
(تقدير) عطلة عيد الفطر                          167
Black Friday                                    139
(تقدير) عيد الأضحى                              113
(تقدير) عطلة عيد الأضحى                          67
عيد ثورة 23 يوليو                                65
عيد العمال                                       63
(تقدير) رأس السنة الهجرية                        63
Valentine's Day                                  59
Mother's Day                                     45
عيد الفصح القبطي                                 40
(تقدير) عيد المولد النبوي                        39
(تقدير) عيد الفطر                                38
شم النسيم; عيد تحرير سيناء                       35
عيد ثورة 25 يناير                                30
شم النسيم                                        30
عيد القوات المسلحة                               23
(تقدير) يوم عرفة                                 20
عيد 

C:\Users\DELL\AppData\Local\Temp\ipykernel_15556\1925827676.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  carSales['Holiday'].fillna('No Holiday', inplace=True)


# **encoding**

In [189]:
def show_unique_values(df):
    for col in df.select_dtypes(include=['object', 'category']).columns:
        print(f"Column: {col}")
        print(f"Unique Count: {df[col].nunique()}")
        print(f"Unique Percentage: {(df[col].nunique() / len(df)) * 100}%")
        print("\n")
show_unique_values(carSales)


Column: Gender
Unique Count: 2
Unique Percentage: 0.00866213348347698%


Column: Company
Unique Count: 30
Unique Percentage: 0.1299320022521547%


Column: Model
Unique Count: 154
Unique Percentage: 0.6669842782277274%


Column: Engine
Unique Count: 2
Unique Percentage: 0.00866213348347698%


Column: Transmission
Unique Count: 2
Unique Percentage: 0.00866213348347698%


Column: Color
Unique Count: 3
Unique Percentage: 0.012993200225215471%


Column: Body Style
Unique Count: 5
Unique Percentage: 0.02165533370869245%


Column: Dealer_Region
Unique Count: 7
Unique Percentage: 0.03031746719216943%


Column: Season
Unique Count: 4
Unique Percentage: 0.01732426696695396%


Column: Holiday
Unique Count: 22
Unique Percentage: 0.09528346831824679%




In [190]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Iterate over all object columns and apply label encoding
for column in ['Engine', 'Transmission', 'Gender', 'Color', 'Dealer_Region','Body Style' ,'Company' , 'Holiday' ]:
  carSales[column] = label_encoder.fit_transform(carSales[column])

In [191]:
mean_map = carSales.groupby('Model')['Price ($)'].mean().to_dict()  
carSales['model'] = carSales['Model'].map(mean_map)
carSales.drop('Model', axis=1, inplace=True)

In [194]:
def show_unique_values(df):
    for col in df.select_dtypes(include=['object', 'category']).columns:
        print(f"Column: {col}")
        print(f"Unique Count: {df[col].nunique()}")
        print(f"Unique Percentage: {(df[col].nunique() / len(df)) * 100}%")
        print(f"the names of the unique values are: {df[col].unique()}")
        print("\n")
show_unique_values(carSales)

Column: Season
Unique Count: 4
Unique Percentage: 0.01732426696695396%
the names of the unique values are: ['Winter' 'Spring' 'Summer' 'Fall']




In [195]:
map_seaonal = {
    'Spring': 0,
    'Summer': 1,
    'Fall': 2,
    'Winter': 3
}
carSales['Season'] = carSales['Season'].map(map_seaonal)


In [196]:
carSales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23089 entries, 0 to 23089
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             23089 non-null  datetime64[ns]
 1   Gender           23089 non-null  int64         
 2   Annual Income    23089 non-null  int64         
 3   Company          23089 non-null  int64         
 4   Engine           23089 non-null  int64         
 5   Transmission     23089 non-null  int64         
 6   Color            23089 non-null  int64         
 7   Price ($)        23089 non-null  int64         
 8   Body Style       23089 non-null  int64         
 9   Dealer_Region    23089 non-null  int64         
 10  Year             23089 non-null  int32         
 11  Month            23089 non-null  int32         
 12  DayOfWeek        23089 non-null  int32         
 13  Season           23089 non-null  int64         
 14  Price_to_Income  23089 non-null  float64   

In [197]:
carSales.columns.all 

<bound method Index.all of Index(['Date', 'Gender', 'Annual Income', 'Company', 'Engine', 'Transmission',
       'Color', 'Price ($)', 'Body Style', 'Dealer_Region', 'Year', 'Month',
       'DayOfWeek', 'Season', 'Price_to_Income', 'Holiday', 'Is_Holiday',
       'Day', 'WeekOfYear', 'Is_Weekend', 'model'],
      dtype='object')>

In [198]:
carSales.head()

,Date,Gender,Annual Income,Company,Engine,Transmission,Color,Price ($),Body Style,Dealer_Region,...,Month,DayOfWeek,Season,Price_to_Income,Holiday,Is_Holiday,Day,WeekOfYear,Is_Weekend,model
0,2022-01-02,1,13500,8,0,0,0,26000,3,4,...,1,6,3,1.925926,11,0,2,52,1,22367.847619
1,2022-01-02,1,1480000,7,0,0,0,19000,3,0,...,1,6,3,0.012838,11,0,2,52,1,22465.016000
2,2022-01-02,1,1035000,4,1,1,2,31500,2,2,...,1,6,3,0.030435,11,0,2,52,1,42132.740909
3,2022-01-02,1,13500,27,1,1,1,14000,3,5,...,1,6,3,1.037037,11,0,2,52,1,26243.834356
4,2022-01-02,1,1465000,0,0,0,2,24500,1,3,...,1,6,3,0.016724,11,0,2,52,1,22819.456274


In [199]:
carSales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23089 entries, 0 to 23089
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             23089 non-null  datetime64[ns]
 1   Gender           23089 non-null  int64         
 2   Annual Income    23089 non-null  int64         
 3   Company          23089 non-null  int64         
 4   Engine           23089 non-null  int64         
 5   Transmission     23089 non-null  int64         
 6   Color            23089 non-null  int64         
 7   Price ($)        23089 non-null  int64         
 8   Body Style       23089 non-null  int64         
 9   Dealer_Region    23089 non-null  int64         
 10  Year             23089 non-null  int32         
 11  Month            23089 non-null  int32         
 12  DayOfWeek        23089 non-null  int32         
 13  Season           23089 non-null  int64         
 14  Price_to_Income  23089 non-null  float64   

In [200]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
carSales = carSales.copy()
carSales[['Annual Income', 'Price ($)']] = scaler.fit_transform(carSales[['Annual Income', 'Price ($)']])


#

## save 

In [104]:
carSales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23089 entries, 0 to 23089
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             23089 non-null  datetime64[ns]
 1   Gender           23089 non-null  int64         
 2   Annual Income    23089 non-null  float64       
 3   Engine           23089 non-null  int64         
 4   Transmission     23089 non-null  int64         
 5   Color            23089 non-null  object        
 6   Price ($)        23089 non-null  float64       
 7   Body Style       23089 non-null  object        
 8   Year             23089 non-null  int32         
 9   Month            23089 non-null  int32         
 10  DayOfWeek        23089 non-null  int32         
 11  Season           23089 non-null  object        
 12  Price_to_Income  23089 non-null  float64       
 13  Day              23089 non-null  int32         
 14  WeekOfYear       23089 non-null  UInt32    

In [202]:
joblib.dump(carSales, 'Datamodeling.pkl')


['Datamodeling.pkl']